 ## Division Class Propensity Recommendation script     
Last update: March 4,2021  
Version 1.2   
**Output format:** 
cmd_id | category | division_id | division |class_id | class | propensity_score | propensity_rank | next_best_rank

In [1]:
from pyspark.sql import functions as F
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder, TrainValidationSplit
from pyspark.ml.feature import StringIndexer
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.sql.types import StructType, StructField, IntegerType,FloatType, StringType
from pyspark.sql.functions import col, rank,concat, concat_ws, monotonically_increasing_id
from pyspark.ml.recommendation import ALS
from pyspark.sql.functions import monotonically_increasing_id

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
1307,application_1570493391423_69074,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
sc = spark.sparkContext
sc.setCheckpointDir("s3://nmg-analytics-ds-prod/ds/dev/Users/nmdst400/checkpointdir")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
#Code Parameters
#brand = 'nm'
#env = 'prod'
#wbrand ='NM'

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
input_f='s3://nmg-analytics-ds-prod/ds/'+env+'/ClientConnect/CMD/data/'+brand+'/cust_division_class_rank.csv.gz'
df=spark.read.option("header","true").csv(input_f, sep=',')
#df.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+--------------------+-----------+--------------------+--------+----------+-----+
| cmd_id|            category|division_id|            division|class_id|     class|score|
+-------+--------------------+-----------+--------------------+--------+----------+-----+
|002vWGm|Contemporary Appa...|         14|Contemporary Apparel|     124|     Dress|    3|
|002vWGm|Designer Handbags...|         35|   Designer Handbags|      29|Top Handle|    3|
|002vWGm|       Jewelry Rings|         56|             Jewelry|      40|     Rings|    5|
|002vWGm|Ladies Shoes Sandals|         34|        Ladies Shoes|      54|   Sandals|    3|
|002vWGm|Women's Apparel G...|         11|     Women's Apparel|     139|     Gowns|    1|
|002wpFg|Contemporary Appa...|         14|Contemporary Apparel|     107|Body Suits|    1|
|002wpFg|Contemporary Appa...|         14|Contemporary Apparel|     124|     Dress|    5|
|002wpFg|Fashion Accessori...|         36| Fashion Accessories|      89|Sunglasses|    3|
|002wpFg|F

In [5]:
df = df.withColumn('product', concat_ws('_','category','division_id','division','class_id','class'))


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
indexer1 = StringIndexer(inputCol="cmd_id", outputCol="userID")
df_staging = indexer1.fit(df).transform(df)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
indexer2 = StringIndexer(inputCol="product", outputCol="catID")
df_staging = indexer2.fit(df_staging).transform(df_staging)
df_1=df_staging.selectExpr('userID','catID','cast(score as double) score')
(train_set, validation_set) = df_1.randomSplit([0.8, 0.2])

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Search optimal model

In [8]:
# Build the recommendation model using ALS on the training data
als = ALS(maxIter=10, rank= 3, regParam=0.05, implicitPrefs=False,coldStartStrategy="drop",\
          userCol='userID', itemCol='catID', ratingCol='score')

model = als.fit(train_set)
#predictions = model.transform(validation_set)

#evaluator = RegressionEvaluator(metricName="mae", labelCol="score", predictionCol="prediction")
#mae = evaluator.evaluate(predictions)

#print("Mean Absolute Error ", mae)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Create the Recommendations for all users

In [9]:
# Generate top 10 recommendations for each user
userRecs = model.recommendForAllUsers(15)

spark.conf.set("spark.sql.execution.arrow.enabled", "true")
userRecs.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- userID: integer (nullable = false)
 |-- recommendations: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- catID: integer (nullable = true)
 |    |    |-- rating: float (nullable = true)

In [10]:
userRecs_output=userRecs.select('userID',F.posexplode('recommendations')).select('userID','col.catID','pos','col.rating').checkpoint()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
userIndex=df_staging.selectExpr('cast(userID as int) userID','cmd_id').distinct()
catIndex=df_staging.selectExpr('cast(catID as int) catID','product').distinct()
recs_df = userRecs_output.join(catIndex,userRecs_output.catID==catIndex.catID).select('userID','product','pos','rating')


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
recs_df = recs_df.join(userIndex,recs_df.userID==userIndex.userID).select('cmd_id','product','pos','rating')
recs_df = recs_df.withColumnRenamed("pos", "propensity_rank").withColumnRenamed("rating","propensity_score")
recs_df = recs_df.withColumn("propensity_rank",F.col("propensity_rank")+1)


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

The prediction scores returned by **ALS** with implicit feedbacks with Apache Spark aren't normalized to fit be between [0,1] or [see](https://stackoverflow.com/questions/46904078/spark-als-recommendation-system-have-value-prediction-greater-than-1). ALS uses stochastic gradient descent and approximations to compute (and re-compute) users and item factors on each step to minimize the cost function which allows it to scale.  
As a matter of fact, normalizing those scores isn't relevant. The reason for this is actually that those scores doesn't mean much on their own. [refer](https://stackoverflow.com/questions/46462470/how-can-i-evaluate-the-implicit-feedback-als-algorithm-for-recommendations-in-ap/46490352#46490352).

### Scaling into range 1-100

In [13]:
max_recs, min_recs = recs_df.select("propensity_score").rdd.max()[0], recs_df.select("propensity_score").rdd.min()[0]
recs_df= recs_df.withColumn("propensity_score",(100-1)*(recs_df["propensity_score"]- min_recs)/(max_recs-min_recs) + 1 )


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Create new key column to join

In [14]:
recs_df = recs_df.withColumn("id_product",F.concat(col('cmd_id'), col('product')))


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…


### Load data recent  historical trans data (default=60 days) 

In [16]:
input_dir='s3://nmg-analytics-ds-prod/ds/'+env+'/ClientConnect/CMD/data/'+brand+'/'
f = input_dir+'recent_history.csv.gz'
df_recent = spark.read.option("header","True").csv(f)


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+----------+-----------+-------------------+-----------+-------------------+--------+----------------+-------+------------------------+
| cmd_id|trans_date|division_id|           division|designer_id|           designer|class_id|           class|  total|days_since_last_purchase|
+-------+----------+-----------+-------------------+-----------+-------------------+--------+----------------+-------+------------------------+
|d0p0gEu|2021-02-26|         34|       Ladies Shoes|      14628|Salvatore Ferragamo|      60|Ballerinas/Flats| 193.55|                      11|
|d0p0gEu|2021-01-27|         81|       Fine Apparel|      11216|      Co Collection|       2|        Sweaters|-389.55|                      41|
|d0p0gEu|2021-01-27|         34|       Ladies Shoes|      15286|         Tory Burch|      61|         Booties|-224.42|                      41|
|d0p0gEu|2021-01-27|         44|              Men's|      15362|                UGG|      52|           Boots|  -81.9|                  

In [17]:
df_recent = df_recent.withColumn('category',concat_ws(' ','division','class'))
df_recent = df_recent.withColumn('product',concat_ws('_','category','division_id','division','class_id','class'))
df_recent = df_recent.withColumn('id_product',F.concat(col('cmd_id'), F.col('product'))).select('cmd_id','product','id_product')


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+--------------------+--------------------+
|  cmd_id|             product|          id_product|
+--------+--------------------+--------------------+
|16WKJv3E|Beauty Makeup_53_...|16WKJv3EBeauty Ma...|
| jaT657n|Ladies Shoes Sand...|jaT657nLadies Sho...|
| t8bmEVs|Men's Sweaters_44...|t8bmEVsMen's Swea...|
| sGLF5Bx|Ladies Shoes Sand...|sGLF5BxLadies Sho...|
| ueWIgYb|Women's Apparel P...|ueWIgYbWomen's Ap...|
| diZwS0c|Women's Apparel S...|diZwS0cWomen's Ap...|
| odQSahC|Beauty Hair_53_Be...|odQSahCBeauty Hai...|
| ts2yycL|Men's Pants_44_Me...|ts2yycLMen's Pant...|
| ZS8y91Q|Beauty Skin Care_...|ZS8y91QBeauty Ski...|
|1Wi49efQ|Ladies Shoes Boot...|1Wi49efQLadies Sh...|
|1BR3y0gX|Beauty Skin Care_...|1BR3y0gXBeauty Sk...|
|10zPfEwe|Contemporary Appa...|10zPfEweContempor...|
| t9hxwwK|Intimate Apparel ...|t9hxwwKIntimate A...|
|1TBmzS71|Beauty Skin Care_...|1TBmzS71Beauty Sk...|
| qzHI2dE|Contemporary Appa...|qzHI2dEContempora...|
| b2ed4ni|Designer Handbags...|b2ed4niDesigner

In [18]:
# find cmd_id with category (in recommendation list) have not purchased before (in 3 year hist trans)
df_recent = recs_df.join(df_recent ,recs_df.id_product == df_recent.id_product, "leftanti")
df_recent = df_recent.selectExpr('cmd_id','product','propensity_score')
#df_recent.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [19]:
# Re_order after merge
df_recent = df_recent.orderBy(F.col("cmd_id"),F.col("propensity_score").desc())
df_recent = df_recent.selectExpr("cmd_id", "product","propensity_score")


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [20]:
from pyspark.sql.window import Window
window = Window.partitionBy(df_recent['cmd_id']).orderBy(df_recent['propensity_score'].desc())
df_recent = df_recent.select('*', F.row_number().over(window).alias('next_best_rank'))


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [21]:
recs_df = recs_df.selectExpr('cmd_id','product','propensity_score','propensity_rank')
recs_df = recs_df.join(df_recent.selectExpr('cmd_id','product','next_best_rank'),['cmd_id','product'],how='left')
recs_df= recs_df.orderBy(F.col("cmd_id"),F.col("propensity_score").desc(),F.col("next_best_rank"))


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+--------------------+------------------+---------------+--------------+
| cmd_id|             product|  propensity_score|propensity_rank|next_best_rank|
+-------+--------------------+------------------+---------------+--------------+
|002meRU|Women's Apparel G...|23.546361604805288|              1|             1|
|002meRU|Women's Apparel D...|22.883442427779137|              2|             2|
|002meRU|Fine Apparel Dres...|21.466008687574693|              3|             3|
|002meRU|Fine Apparel Shir...| 21.18152214941159|              4|             4|
|002meRU|Fine Apparel Pant...| 21.01865710440034|              5|             5|
|002meRU|Fine Apparel Swea...|20.591663651852397|              6|             6|
|002meRU|Fine Apparel Jack...| 20.17916921810682|              7|             7|
|002meRU|Women's Apparel J...| 19.55659219112099|              8|             8|
|002meRU|Women's Apparel S...|19.540087111407527|              9|             9|
|002meRU|Ladies Shoes Snea..

In [22]:
df_recent = recs_df.select(F.split(recs_df.product,"_")).rdd.flatMap(
              lambda x: x).toDF(schema=["col1","col2","col3","col4","col5"])

df_recent = df_recent.withColumn("id", monotonically_increasing_id())


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [23]:
recs_df = recs_df.withColumn("id", monotonically_increasing_id())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [24]:
recs_df = recs_df.join(df_recent, df_recent.id== recs_df.id).drop("id")
recs_df = recs_df.withColumnRenamed('col1','category').withColumnRenamed('col2','division_id').withColumnRenamed('col3','division').withColumnRenamed('col4','class_id').withColumnRenamed('col5','class')
recs_df = recs_df.orderBy(F.col("cmd_id"),F.col("propensity_score").desc(),F.col("next_best_rank"))


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [25]:
recs_df = recs_df.select('cmd_id','category','division_id','division','class_id','class','propensity_score','propensity_rank','next_best_rank')
#recs_df.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [26]:
#recs_df.filter(recs_df.cmd_id=='d0p0gEu').show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### Saving to S3

In [27]:

recs_df.write.parquet("s3a://nmg-analytics-ds-prod/ds/{0}/product_propensity/{1}/output/division_class_propensity/".format(env,brand),mode="overwrite")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…